In [44]:
import random

def replace_random_integer_with_letter(nested_list, letter='X'):
    """
    Recursively traverses a nested list and replaces a randomly chosen integer with a letter.
    
    Parameters:
    nested_list (list): The nested list to process.
    letter (str): The letter to replace an integer with. Defaults to 'X'.
    
    Returns:
    None: The function modifies the nested list in place.
    """
    
    # Helper function to count integers
    def count_integers(lst):
        count = 0
        for item in lst:
            if isinstance(item, list):
                count += count_integers(item)
            elif isinstance(item, int):
                count += 1
        return count
    
    # Helper function to replace the nth integer
    def replace_nth_integer(lst, n, letter):
        for i, item in enumerate(lst):
            if isinstance(item, list):
                n = replace_nth_integer(item, n, letter)
                if n == 0:
                    return 0  # Early return if replacement is done
            elif isinstance(item, int):
                n -= 1
                if n == 0:
                    lst[i] = letter
                    return 0
        return n
    
    total_integers = count_integers(nested_list)
    random_integer_position = random.randint(1, total_integers)
    
    replace_nth_integer(nested_list, random_integer_position, letter)

# Example usage
nested_list = [[1, 2, [3, 4]], [5, [6, 7]]]
replace_random_integer_with_letter(nested_list)
print(nested_list)

[[1, 2, [3, 4]], [5, ['X', 7]]]


Testing logic with molecules.

In [81]:
%reload_ext autoreload
%autoreload 2

import rdkit
rdkit.Chem.Draw.IPythonConsole.ipython_maxProperties = -1

import dgym as dg

# load all data
path = '../../dgym-data'

deck = dg.MoleculeCollection.load(
    f'{path}/DSi-Poised_Library_annotated.sdf',
    reactant_names=['reagsmi1', 'reagsmi2', 'reagsmi3']
)

reactions = dg.ReactionCollection.from_json(
    path = f'{path}/All_Rxns_rxn_library.json',
    smarts_col = 'reaction_string',
    classes_col = 'functional_groups'
)

building_blocks = dg.datasets.disk_loader(f'{path}/Enamine_Building_Blocks_Stock_262336cmpd_20230630.sdf')
fingerprints = dg.datasets.fingerprints(f'{path}/Enamine_Building_Blocks_Stock_262336cmpd_20230630_atoms.fpb')

import torch
import pyarrow.parquet as pq
table = pq.read_table('../../dgym-data/sizes.parquet')[0]
sizes = torch.tensor(table.to_numpy())

In [46]:
from dgym.molecule import Molecule
from dgym.envs.designer import Designer, Generator
from dgym.envs.drug_env import DrugEnv
from dgym.agents import SequentialDrugAgent
from dgym.agents.exploration import EpsilonGreedy
from dgym.experiment import Experiment

designer = Designer(
    Generator(building_blocks, fingerprints, sizes),
    reactions,
    cache = True
)

In [87]:
designer.reset_cache()
res = designer.design(deck[1], 1, mode='grow')

In [93]:
def replace_reactants(molecule):
    
    if not molecule.reactants:
        return molecule
    
    replace_reactants(molecule)
    
    return molecule
# res[0].reactants[1]